In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
import keras
 
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Model, layers

from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve
import matplotlib.pyplot as plt
 
print(tf.__version__)

2.5.0


## Get Dataset and prepare

You can get mnist data set for handwritter numbers from 0-9 in gray scale with square dimension 28*28 with total number of images about 60000 image.

In [ ]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data() # Load data x is the images and y are lables

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
batch_size =32
num_classes = 10 # the number of output classes to make classification between them 0-9 in this example
learning_rate = 0.0001 # always from 0-1
training_steps = 200
display_step = 10

# Network parameters.
conv1_filters = 32 # number of filters for 1st conv layer.
conv2_filters = 64 # number of filters for 2nd conv layer.
fc1_units = 1024 # number of neurons for 1st fully-connected layer.

In [ ]:
print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))

Train: X=(60000, 28, 28), y=(60000,)
Test: X=(10000, 28, 28), y=(10000,)


In [ ]:
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [ ]:
# Use tf.data API to shuffle and batch data.
#train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:
y_train = y_train[:60000]
y_test =y_test[:10000]

x_train = tf.reshape(x_train, [-1, 28, 28, 1])
x_test = tf.reshape(x_test, [-1, 28, 28, 1])

print(x_train.shape)


(60000, 28, 28, 1)


In [ ]:
# Define a simple sequential model
def create_model():

  model = tf.keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=5, activation=tf.nn.relu, input_shape=(28,28,1)),
    keras.layers.MaxPool2D(2, strides=2),
    keras.layers.Conv2D(64, kernel_size=5, activation=tf.nn.relu),
    keras.layers.MaxPool2D(2, strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(1024),
    keras.layers.Dropout(rate=0.5),
    keras.layers.Dense(10)

  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])
  
  model.build((60000, 28, 28, 1)) # this line wasn't added so an error was launched

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (60000, 24, 24, 32)       832       
_________________________________________________________________
module_wrapper_1 (ModuleWrap (60000, 12, 12, 32)       0         
_________________________________________________________________
module_wrapper_2 (ModuleWrap (60000, 8, 8, 64)         51264     
_________________________________________________________________
module_wrapper_3 (ModuleWrap (60000, 4, 4, 64)         0         
_________________________________________________________________
module_wrapper_4 (ModuleWrap (60000, 1024)             0         
_________________________________________________________________
module_wrapper_5 (ModuleWrap (60000, 1024)             1049600   
_________________________________________________________________
module_wrapper_6 (ModuleWrap (60000, 1024)             0

In [ ]:
from google.colab import drive
drive.mount('/ssd_keras')
import os
os.chdir("/ssd_keras/My Drive/Colab_Notebooks")

Mounted at /ssd_keras


In [ ]:

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="/ssd_keras/MyDrive/Colab_Notebooks/cp.ckpt",
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(x_train, 
          y_train,  
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[cp_callback])  # Pass callback to training



Epoch 1/10
1875/1875 [==============================] - 81s 43ms/step - loss: 0.1151 - sparse_categorical_accuracy: 0.9648 - val_loss: 0.0360 - val_sparse_categorical_accuracy: 0.9884

Epoch 00001: saving model to /ssd_keras/MyDrive/Colab_Notebooks/cp.ckpt
Epoch 2/10
1875/1875 [==============================] - 80s 43ms/step - loss: 0.0445 - sparse_categorical_accuracy: 0.9867 - val_loss: 0.0461 - val_sparse_categorical_accuracy: 0.9856

Epoch 00002: saving model to /ssd_keras/MyDrive/Colab_Notebooks/cp.ckpt
Epoch 3/10
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0341 - sparse_categorical_accuracy: 0.9891 - val_loss: 0.0419 - val_sparse_categorical_accuracy: 0.9865

Epoch 00003: saving model to /ssd_keras/MyDrive/Colab_Notebooks/cp.ckpt
Epoch 4/10
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0296 - sparse_categorical_accuracy: 0.9907 - val_loss: 0.0357 - val_sparse_categorical_accuracy: 0.9901

Epoch 00004: saving model to /ssd_keras/MyDr

## Evaluation the model

In [ ]:
model = create_model()

model.load_weights("/ssd_keras/MyDrive/Colab_Notebooks/cp.ckpt")

## using the usual accuracy and loss
loss, acc = model.evaluate(x_test, y_test, verbose=2)
## using confusion matricx
mnist_pred = model.predict_classes(x_test)
print("confusion matrix", confusion_matrix(y_test, mnist_pred))
print("precision score",precision_score(y_test, mnist_pred, average= None))
print("recall score", recall_score(y_test, mnist_pred, average=None))


313/313 - 3s - loss: 0.0732 - sparse_categorical_accuracy: 0.9876


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


confusion matrix [[ 974    1    1    0    0    1    0    1    2    0]
 [   0 1128    0    3    0    2    1    0    1    0]
 [   0    2 1020    3    0    1    0    4    2    0]
 [   0    0    0 1004    0    4    0    1    1    0]
 [   0    2    1    0  974    0    2    1    1    1]
 [   1    0    0    8    0  881    1    1    0    0]
 [   7    1    0    1    1    2  944    0    2    0]
 [   0    4    3    2    2    0    0 1014    1    2]
 [   0    0    0    2    0    0    0    0  970    2]
 [   1    0    1    1   13    9    0   10    7  967]]
precision score [0.99084435 0.99121265 0.99415205 0.98046875 0.98383838 0.97888889
 0.99578059 0.98255814 0.98277609 0.99485597]
recall score [0.99387755 0.9938326  0.98837209 0.99405941 0.99185336 0.98766816
 0.98538622 0.98638132 0.99589322 0.95837463]
